In [82]:
%pip install Sastrawi

import pandas as pd
import re
import nltk
import math
import sys
import time
import csv
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from IPython.display import clear_output
from datetime import datetime

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\Candra\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [83]:
FOLDER_LOG_RIWAYAT = 'Riwayat'
FILE_LOG_RIWAYAT = os.path.join(FOLDER_LOG_RIWAYAT, 'riwayat_proyek.csv')

def log_event_ke_riwayat(kategori, keterangan):
    try:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        data_row = [timestamp, kategori, keterangan]
        file_exists = os.path.isfile(FILE_LOG_RIWAYAT)
        with open(FILE_LOG_RIWAYAT, mode='a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            if not file_exists:
                writer.writerow(['timestamp', 'kategori_event', 'keterangan'])
            writer.writerow(data_row)
    except Exception as e:
        print(f"⚠️ GAGAL mencatat riwayat korpus: {e}")

NAMA_FOLDER_KORPUS = 'Documents'

semua_file_korpus = []
try:
    # Cari semua file CSV di dalam folder korpus
    for filename in os.listdir(NAMA_FOLDER_KORPUS):
        if filename.endswith('.csv'):
            semua_file_korpus.append(os.path.join(NAMA_FOLDER_KORPUS, filename))
    
    if not semua_file_korpus:
        print(f"❌ ERROR: Tidak ada file .csv ditemukan di folder '{NAMA_FOLDER_KORPUS}'.")
        df_corpus = pd.DataFrame() # Buat dataframe kosong
    else:
        print(f"ℹ️ Ditemukan {len(semua_file_korpus)} file korpus:")
        for f in semua_file_korpus:
            print(f"  - {f}")
        
        # Baca dan gabungkan semua file CSV menjadi satu DataFrame
        list_df = [pd.read_csv(file) for file in semua_file_korpus]
        df_corpus = pd.concat(list_df, ignore_index=True)
        
        # (Opsional tapi disarankan) Hapus duplikat total jika ada
        df_corpus = df_corpus.drop_duplicates()
        
        print(f"\n✅ SUKSES: Semua file korpus telah digabungkan.")
        print(f"   Total Baris (Ulasan) Ditemukan: {len(df_corpus)}")
        
        log_event_ke_riwayat(
            kategori='Pemuatan Korpus', 
            keterangan=f'Memuat total {len(df_corpus)} ulasan dari {len(semua_file_korpus)} file.'
        )
        
        print(df_corpus.head())

except FileNotFoundError:
    print(f"❌ ERROR: Folder '{NAMA_FOLDER_KORPUS}' tidak ditemukan.")
    df_corpus = pd.DataFrame() # Buat dataframe kosong
except Exception as e:
    print(f"❌ ERROR saat memuat korpus: {e}")

ℹ️ Ditemukan 1 file korpus:
  - Documents\corpus_master.csv

✅ SUKSES: Semua file korpus telah digabungkan.
   Total Baris (Ulasan) Ditemukan: 299
   Doc_ID         Nama_Tempat                      Lokasi  Rating  \
0       1  Kuncen Camp Ground  Kab. Semarang, Jawa Tengah     5.0   
1       2  Kuncen Camp Ground  Kab. Semarang, Jawa Tengah     5.0   
2       3  Kuncen Camp Ground  Kab. Semarang, Jawa Tengah     5.0   
3       4  Kuncen Camp Ground  Kab. Semarang, Jawa Tengah     5.0   
4       5  Kuncen Camp Ground  Kab. Semarang, Jawa Tengah     5.0   

                                         Teks_Mentah  
0  Bagus banget tempatnya, terkonsep dan guide ny...  
1  Sangat menyenangkan untuk camping ceria.\r\nNy...  
2  Tempatnya asri dan sejuk, sudah lumayan ramai ...  
3  3 kali ke sini, sekali ikut acara, 2 kali biki...  
4  Tempat yang cocok untuk acara kemah, kami kema...  


In [84]:
# Import Sastrawi
try:
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
except ImportError:
    print("Warning: Sastrawi is not available. Using Dummy Stemmer.")
    class DummyStemmer:
        def stem(self, text):
            return text
    class StemmerFactory:
        def create_stemmer(self):
            return DummyStemmer()

# Safety for NLTK Stopwords      
try:
    from nltk.corpus import stopwords
    stopwords_id = set(stopwords.words('indonesian')) 
except (LookupError, ImportError):
    print("Warning: Indonesian stopwords failed to load. Using minimal manual list.")
    stopwords_id = {"yang", "dan", "di", "ke", "adalah", "dengan", "saya", "ini"}

negation_words = ['tidak', 'kurang', 'jangan', 'bukan', 'tanpa']
for word in negation_words:
    if word in stopwords_id:
        stopwords_id.remove(word)
        print(f"'{word}' dihapus dari stopwords untuk menangani negasi.")

# Inisialisasi Tools
stemmer = StemmerFactory().create_stemmer()

'tidak' dihapus dari stopwords untuk menangani negasi.
'kurang' dihapus dari stopwords untuk menangani negasi.
'jangan' dihapus dari stopwords untuk menangani negasi.
'bukan' dihapus dari stopwords untuk menangani negasi.
'tanpa' dihapus dari stopwords untuk menangani negasi.


In [85]:
def load_map_from_csv(filepath):
    """
    Memuat file CSV (kolom A: key, kolom B: value) ke dalam dictionary.
    Fungsi ini mengabaikan baris yang diawali '#' (untuk komentar)
    dan mengabaikan kolom ekstra (seperti kolom 'kategori').
    """
    try:
        # 'comment=#' memberi tahu pandas untuk mengabaikan baris yang diawali #
        df = pd.read_csv(filepath, comment='#')
        
        # Ambil nama kolom pertama (A) dan kedua (B)
        key_col = df.columns[0]
        value_col = df.columns[1]
        
        # Hapus baris yang mungkin kosong di kolom A atau B
        df = df.dropna(subset=[key_col, value_col])
        
        # Buat dictionary: Kolom A jadi key, Kolom B jadi value
        mapper_dict = pd.Series(df[value_col].values, index=df[key_col]).to_dict()
        
        print(f"Berhasil memuat {len(mapper_dict)} aturan dari {filepath}")
        return mapper_dict
        
    except FileNotFoundError:
        print(f"!!! PERINGATAN: File konfigurasi {filepath} tidak ditemukan. Menggunakan dictionary kosong.")
        return {}
    except Exception as e:
        print(f"!!! ERROR saat memuat {filepath}: {e}")
        return {}

In [86]:
# Definisi Pemegangan Frasa Kompleks
def substitute_complex_phrases(text, phrase_map):
    """Mengganti frasa kompleks dengan single token sebelum tokenisasi."""
    # Pastikan substitusi dilakukan pada teks lowercase
    text_lower = text.lower()

    sorted_phrases = sorted(phrase_map.items(), key=lambda item: len(item[0]), reverse=True)
    
    for phrase, token in sorted_phrases:
        try:
            # re.escape() mengamankan frasa jika mengandung karakter regex
            regex_phrase = r'\b' + re.escape(str(phrase)) + r'\b'
            
            # Ganti hanya frasa yang ditemukan sebagai "kata utuh"
            text_lower = re.sub(regex_phrase, str(token), text_lower)
            
        except re.error:
            # Fallback ke replace() biasa jika ada error regex
            text_lower = text_lower.replace(str(phrase), str(token))
        
    return text_lower

# Muat PHRASE_MAP dari file eksternal
PHRASE_MAP = load_map_from_csv('Kamus\config_phrase_map.csv')

Berhasil memuat 116 aturan dari Kamus\config_phrase_map.csv


In [87]:
# Muat REGION_MAP dari file eksternal
REGION_MAP = load_map_from_csv('Kamus\config_region_map.csv')

def detect_region_and_filter_query(query_text):
    """
    Menganalisis query untuk menentukan apakah mengandung niat regional.
    Mengembalikan query yang sudah difilter (tanpa kata regional) dan kode region.
    """
    
    query_text_lower = query_text.lower()
    detected_region = None
    
    # Deteksi Region
    for term, region in REGION_MAP.items():
        if term in query_text_lower:
            detected_region = region
            query_text_lower = query_text_lower.replace(term, '') # Hapus kata regional
            # Break setelah region pertama terdeteksi (asumsi hanya 1 region per query)
            break 
            
    # Rebuild Query tanpa kata regional (untuk VSM)
    # Hapus spasi berlebihan dan filter token kosong
    filtered_query_text = " ".join([word for word in query_text_lower.split() if word])
    
    return filtered_query_text, detected_region

Berhasil memuat 32 aturan dari Kamus\config_region_map.csv


In [88]:
# Muat SPECIAL_INTENT_MAP dari file eksternal
SPECIAL_INTENT_MAP = load_map_from_csv('Kamus\config_special_intent.csv')

def detect_intent(query_text):
    """
    Menganalisis query untuk menentukan niat khusus (ALL/RATING).
    Mengembalikan query VSM yang sudah bersih, dan special_intent.
    """
    query_text_lower = query_text.lower()
    special_intent = None
    
    # 1. Deteksi Niat Khusus (ALL/RATING)
    for term, intent in SPECIAL_INTENT_MAP.items():
        if term in query_text_lower:
            special_intent = intent
            query_text_lower = query_text_lower.replace(term, '')
            break
            
    # 3. Rebuild Query VSM
    filtered_query_text = " ".join([word for word in query_text_lower.split() if word])
    
    return filtered_query_text, special_intent

Berhasil memuat 16 aturan dari Kamus\config_special_intent.csv


In [89]:
def analyze_full_query(query_text):
    """
    Menganalisis query untuk intent, region, dan teks VSM terakhir.
    """
    
    # 1. Deteksi Intent
    # Ini membersihkan query dari frasa intent, misal "tempat kemah terbaik"
    query_after_intent, special_intent = detect_intent(query_text)
    
    # 2. Deteksi Region
    # Ini membersihkan query dari frasa region, misal "jawa tengah"
    final_vsm_text, region_filter = detect_region_and_filter_query(query_after_intent)
    
    # 3. Preprocessing Teks VSM
    vsm_tokens = full_preprocessing(final_vsm_text)

    # Tambahan pengecekan khusus jika ada filter region
    if region_filter:
        # Gunakan kata yang sudah di-stem
        generic_fluff_words = {'cari', 'tampil', 'lihat', 'berikan', 'saran', 'rekomendasikan'} 
        
        # Cek apakah vsm_tokens HANYA berisi kata-kata fluff
        if vsm_tokens and all(token in generic_fluff_words for token in vsm_tokens):
            vsm_tokens = [] # Kosongkan token, jangan cari VSM
    
    # Jika token kosong setelah semua filter (misal query-nya hanya "terbaik di jogja")
    if not vsm_tokens and (special_intent or region_filter):
         # Beri kata kunci default agar VSM tidak error
        vsm_tokens = ['kemah'] 

        # Jika intent awalnya kosong TAPI region ada,
        # berarti pengguna HANYA ingin filter region. Ubah intent ke 'ALL'.
        if not special_intent and region_filter:
            special_intent = 'ALL'
        
    return vsm_tokens, special_intent, region_filter

In [90]:
# --- 2. DEFENISI FUNGSI HELPER & VSM CLASSES ---
# Fungsi Pembersihan Karakter Spesial
def remove_special_characters(text):
    if not isinstance(text, str):
        return "" 
    regex = re.compile(r'[^a-zA-Z0-9\s]')
    return re.sub(regex, '', text)

# Fungsi Proses Penuh (Preprocessing)
def full_preprocessing(text):
    if not isinstance(text, str):
        return []
        
    cleaned_text = remove_special_characters(text)
    cleaned_text = re.sub(r'\d', '', cleaned_text)

    text_with_phrases = substitute_complex_phrases(cleaned_text, PHRASE_MAP)
    
    # Simple Tokenization (split by whitespace) & Lowercasing
    words = text_with_phrases.lower().split()
    
    words = [w for w in words if w not in stopwords_id]
    
    # Stemming
    stemmed_words = [stemmer.stem(w) for w in words]
    
    final_words = [w for w in stemmed_words if len(w) > 1]
    return final_words

# Inverted Index Classes
class Node:
    def __init__(self, docId, freq=None):
        self.freq = freq # TF-IDF weight
        self.doc = docId
        self.nextval = None

class SlinkedList:
    def __init__(self, head=None):
        self.head = head

In [91]:
# --- 3. APLIKASI PREPROCESSING & HITUNG DF & IDF (INDEXING PHASE 1) ---
# Pastikan dataset sudah dimuat di df_corpus
df_corpus['Teks_Mentah'] = df_corpus['Teks_Mentah'].fillna('')
df_corpus['Clean_Tokens'] = df_corpus['Teks_Mentah'].apply(full_preprocessing)

N = len(df_corpus)
df_counts = {} # Document Frequency

for tokens in df_corpus['Clean_Tokens']:
    for word in set(tokens): 
        df_counts[word] = df_counts.get(word, 0) + 1

idf_scores = {}
for term, count in df_counts.items():
    idf_scores[term] = math.log10(N / count)

In [92]:
# --- 4. BUILDING THE INVERTED INDEX WITH TF-IDF (INDEXING PHASE 2) ---
linked_list_data = {}
unique_words_all = set(df_counts.keys())

for word in unique_words_all:
    linked_list_data[word] = SlinkedList()
    linked_list_data[word].head = Node(docId=0, freq=None) 

for index, row in df_corpus.iterrows():
    doc_id = row['Doc_ID']
    tokens = row['Clean_Tokens']
    
    tf_in_doc = {}
    for word in tokens:
        tf_in_doc[word] = tf_in_doc.get(word, 0) + 1

    for term, tf in tf_in_doc.items():
        tfidf = tf * idf_scores[term]
        
        linked_list = linked_list_data[term].head
        while linked_list.nextval is not None:
            linked_list = linked_list.nextval
        
        linked_list.nextval = Node(docId=doc_id, freq=tfidf)

# Mapping Doc ID to Name and Rating for final result
df_metadata = df_corpus[['Doc_ID', 'Nama_Tempat', 'Lokasi', 'Rating']].copy()
avg_rating_per_place = df_metadata.groupby('Nama_Tempat')['Rating'].mean().reset_index()
avg_rating_per_place.rename(columns={'Rating': 'Avg_Rating'}, inplace=True)
df_metadata = df_metadata.merge(avg_rating_per_place, on='Nama_Tempat', how='left')
df_metadata.set_index('Doc_ID', inplace=True)

In [93]:
# --- 5. FUNGSI VSM RANKING MURNI ---
def search_by_keyword(query_tokens, special_intent, region_filter):
    """
    Melakukan pencarian berdasarkan token VSM, intent, dan filter region.
    """
    # Tangani kasus special_intent 'ALL'
    if special_intent == 'ALL':
        
        # 1. Ambil semua tempat unik langsung dari metadata
        df_unique_places = df_metadata[['Nama_Tempat', 'Lokasi', 'Avg_Rating']].drop_duplicates(subset='Nama_Tempat').copy()
        
        # 2. Terapkan filter regional jika ada
        if region_filter:
            # Menggunakan .str.contains() untuk mencocokkan substring (misal: 'diy' atau 'semarang')
            df_unique_places = df_unique_places[df_unique_places['Lokasi'].str.lower().str.contains(region_filter, na=False)]

        # 3. Urutkan berdasarkan Rating Tertinggi (sebagai default untuk 'ALL')
        df_unique_places = df_unique_places.sort_values(by='Avg_Rating', ascending=False)
        
        # 4. Ubah format ke dictionary standar
        final_recommendations = []
        for _, row in df_unique_places.iterrows():
            final_recommendations.append({
                'name': row['Nama_Tempat'],
                'location': row['Lokasi'],
                'avg_rating': row['Avg_Rating'],
                'top_vsm_score': 0.0  # Skor VSM 0.0 karena VSM tidak digunakan
            })
        
        # Langsung kembalikan hasilnya
        return final_recommendations

    # 1. Preprocessing Query
    if not query_tokens:
        return []
    
    # 2. Query Vectorization (TF-IDF)
    query_tf = {}
    for word in query_tokens:
        query_tf[word] = query_tf.get(word, 0) + 1
        
    query_weights = {}
    involved_docs = set()
    
    for term, tf in query_tf.items():
        if term in idf_scores:
            query_weights[term] = tf * idf_scores[term]
            
            # Collect all documents involved from the Index
            current_node = linked_list_data[term].head.nextval
            while current_node is not None:
                involved_docs.add(current_node.doc)
                current_node = current_node.nextval
        else:
            continue

    if not involved_docs:
        return []

    # 3. Cosine Similarity (Dot Product Only)
    doc_scores = {doc_id: 0 for doc_id in involved_docs}
    
    # Calculate DOT PRODUCT: Sum(W(t,d) * W(t,q))
    for term, W_q in query_weights.items():
        current_node = linked_list_data[term].head.nextval
        while current_node is not None:
            doc_id = current_node.doc
            W_d = current_node.freq # TF-IDF weight W(t,d)
            doc_scores[doc_id] += W_d * W_q
            current_node = current_node.nextval
            
    # 4. Ranking Ulasan (Doc ID)
    ranked_results_by_doc = sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)
    
    # 5. Agregasi ke Nama Tempat (Mengambil ulasan paling relevan per tempat)
    final_recommendations = []
    unique_names = set()
    
    for doc_id, vsm_score in ranked_results_by_doc:
        try:
            meta = df_metadata.loc[doc_id]
        except KeyError:
            continue
        
        # Filter berdasarkan region jika diminta (opsional)
        if region_filter:
            if region_filter not in meta['Lokasi'].lower():
                continue # Skip dokumen yang tidak sesuai region
            
        name = meta['Nama_Tempat']
        
        if name not in unique_names:
            unique_names.add(name)
            final_recommendations.append({
                'name': name,
                'location': meta['Lokasi'],
                'avg_rating': meta['Avg_Rating'],
                'top_vsm_score': vsm_score
            })

    # 6. Logika Intent
    # Terapkan sorting berdasarkan intent setelah VSM selesai
    if special_intent == 'RATING_TOP':
        # Urutkan berdasarkan Avg_Rating (Tertinggi ke Terendah)
        final_recommendations.sort(key=lambda x: x['avg_rating'], reverse=True)
    
    elif special_intent == 'RATING_BOTTOM':
        # Urutkan berdasarkan Avg_Rating (Terendah ke Tertinggi)
        final_recommendations.sort(key=lambda x: x['avg_rating'], reverse=False)
            
    return final_recommendations

In [94]:
# --- 6. FUNGSI UNTUK RIWAYAT PENCARIAN ---

NAMA_FOLDER_RIWAYAT = 'Riwayat'
NAMA_FILE_RIWAYAT = os.path.join(NAMA_FOLDER_RIWAYAT, 'riwayat_pencarian.csv')

# Definisikan header untuk file CSV
HEADER_RIWAYAT = ['timestamp', 'query_mentah', 'vsm_tokens_final', 'intent_terdeteksi', 'region_terdeteksi']

def log_pencarian(query, tokens, intent, region):
    """Menyimpan detail pencarian ke file CSV."""
    
    try:
        # 1. Dapatkan waktu saat ini
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # 2. Ubah daftar token menjadi string agar mudah dibaca
        # misal: ['kamar', 'mandi'] -> "kamar mandi"
        tokens_str = ' '.join(tokens)
        
        # 3. Siapkan baris data yang akan ditulis
        data_row = [timestamp, query, tokens_str, str(intent), str(region)]
        
        # 4. Cek apakah file sudah ada (untuk menentukan perlu header atau tidak)
        file_exists = os.path.isfile(NAMA_FILE_RIWAYAT)
        
        # 5. Buka file dalam mode 'append' (a)
        with open(NAMA_FILE_RIWAYAT, mode='a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            
            # Jika file baru dibuat, tulis headernya dulu
            if not file_exists:
                writer.writerow(HEADER_RIWAYAT)
                
            # Tulis baris data pencarian
            writer.writerow(data_row)
            
    except Exception as e:
        # Cetak peringatan jika gagal, tapi jangan hentikan program
        print(f"\n!!! PERINGATAN: Gagal menyimpan riwayat pencarian: {e}")

In [95]:
print("==================================================")
print("MESIN PENCARIAN REKOMENDASI TEMPAT KEMAH VSM SIAP!")
print("==================================================")
print("Anda dapat memasukkan kata kunci untuk mencari rekomendasi.")

while True:
    try:    
        # Mengambil input query dari pengguna
        query_text = input("\nMasukkan kata kunci pencarian (atau ketik 'keluar' untuk berhenti): \n").strip()
        
        if query_text.lower() in ('keluar', 'exit', 'berhenti', 'quit', 'stop', 'kembali'):
            print("\nSesi pencarian diakhiri. Terima kasih!")
            break
        
        if not query_text:
            continue

        start_time = time.time()
            
        # Panggil fungsi pencarian VSM
        vsm_tokens, intent, region = analyze_full_query(query_text)

        # Simpan riwayat pencarian
        log_pencarian(query_text, vsm_tokens, intent, region)
        
        # Panggil fungsi pencarian dengan hasil analisis
        vsm_ranking = search_by_keyword(vsm_tokens, intent, region)

        # Deteksi region untuk ditampilkan
        filter_status = f"Filtered by: {region.upper()}" if region else "No Region Filter Applied"
        intent_status = f"Intent: {intent}" if intent else "Intent: VSM Relevancy"
        
        print("\n--------------------------------------------------")
        print(f"HASIL PENCARIAN untuk: '{query_text}'")
        print(f"Kata Kunci Diproses: {vsm_tokens}")
        print(f"Status Filter: {filter_status}")
        print(f"Mode Pencarian: {intent_status}")
        print("--------------------------------------------------")

        if vsm_ranking:
            if intent == 'RATING_TOP':
                print("Rekomendasi Tempat Kemah (Diurutkan berdasarkan Rating Tertinggi):")
            elif intent == 'RATING_BOTTOM':
                print("Rekomendasi Tempat Kemah (Diurutkan berdasarkan Rating Terendah):")
            else:
                print("Rekomendasi Tempat Kemah (Diurutkan berdasarkan Relevansi Ulasan):")
            
            for i, item in enumerate(vsm_ranking):
                print(f"{i+1}. {item['name']}")
                print(f"   | Lokasi: {item['location']}")
                print(f"   | Rata-rata Rating Tempat: {item['avg_rating']:.2f}")
                print(f"   | Skor Relevansi (VSM Score): {item['top_vsm_score']:.4f}")
            
            # Logika lanjut
            continue_input = input("\nApakah Anda ingin melanjutkan pencarian? (ya/tidak): ").strip().lower()
            
            if continue_input not in ('ya', 'y'):
                print("\nSesi pencarian diakhiri. Terima kasih!")
                break
                
            # Hapus output sebelum loop selanjutnya
            clear_output(wait=True) 
            print("Mesin pencarian siap untuk query selanjutnya...")
        else:
            print("Tidak ditemukan tempat kemah yang relevan dengan kata kunci ini.")
            
            # Logika lanjut
            continue_input = input("\nApakah Anda ingin melanjutkan pencarian? (ya/tidak): ").strip().lower()
            
            if continue_input not in ('ya', 'y'):
                print("\nSesi pencarian diakhiri. Terima kasih!")
                break
                
            # Hapus output sebelum loop selanjutnya
            clear_output(wait=True) 
            print("Mesin pencarian siap untuk query selanjutnya...")

    except KeyboardInterrupt:   
        print("\nSesi pencarian diakhiri oleh pengguna. Terima kasih!")
        break

MESIN PENCARIAN REKOMENDASI TEMPAT KEMAH VSM SIAP!
Anda dapat memasukkan kata kunci untuk mencari rekomendasi.

--------------------------------------------------
HASIL PENCARIAN untuk: 'tempat kemah terbaik di jateng'
Kata Kunci Diproses: ['kemah']
Status Filter: Filtered by: JAWA TENGAH
Mode Pencarian: Intent: RATING_TOP
--------------------------------------------------
Rekomendasi Tempat Kemah (Diurutkan berdasarkan Rating Tertinggi):
1. Ratan Lurung Basecamp Gedongsongo
   | Lokasi: Kab. Semarang, Jawa Tengah
   | Rata-rata Rating Tempat: 4.80
   | Skor Relevansi (VSM Score): 0.9051
2. Kuncen Camp Ground
   | Lokasi: Kab. Semarang, Jawa Tengah
   | Rata-rata Rating Tempat: 4.80
   | Skor Relevansi (VSM Score): 0.4525
3. Telaga Cebong
   | Lokasi: Wonosobo, Jawa Tengah
   | Rata-rata Rating Tempat: 4.71
   | Skor Relevansi (VSM Score): 0.6788
4. Camp Ground Bukit Sikunir
   | Lokasi: Wonosobo, Jawa Tengah
   | Rata-rata Rating Tempat: 4.60
   | Skor Relevansi (VSM Score): 1.3576
5.

In [96]:
import joblib
import os

# Buat folder untuk menyimpan aset
os.makedirs('Assets', exist_ok=True)

# Simpan Inverted Index, IDF Scores, dan Metadata
joblib.dump(linked_list_data, 'Assets/linked_list_data.pkl')
joblib.dump(idf_scores, 'Assets/idf_scores.pkl')
joblib.dump(df_metadata, 'Assets/df_metadata.pkl')

print("✅ Aset indeks VSM berhasil disimpan di folder 'Assets/'.")

✅ Aset indeks VSM berhasil disimpan di folder 'Assets/'.
